In [1]:
import os
import numpy as np
import pandas as pd
import time
from datetime import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.lines as mpl_lines
%matplotlib inline
import tables
import socket
import tables

In [2]:
dt.fromtimestamp(int(1459196891.412194)).strftime("%Y-%m-%d %H:%M")
def pd_timestamp():
    t = dt.utcnow()
    return pd.Timestamp(np.datetime64(dt.utcnow()))
    #return time.mktime(t.timetuple())+(t.microsecond/1e6)

## Example data frame

In [656]:
# example data to work with
df = pd.read_csv('../local/db/consolidation/bed_2016-03-26.csv', names=['timestamp', 'sid', 'value'])
df.timestamp = df.timestamp.astype('datetime64[s]')
df.value = df.value.astype(np.float16)
df.set_index('timestamp', inplace=True)
df = df.drop(['timestamp'])
df = df.tz_localize('UTC').tz_convert('Europe/Amsterdam')
# df.type.loc[df.type=='strain'] = 7
# df.type.loc[df.type=='temp'] = 8
# df.type.loc[df.type=='motion'] = 9;
# df.type.astype('uint8');
df.sid = df.sid.astype('category')
# df.sid.cat.rename_categories([1,2,3]) # not inplace
df.sid.cat.categories = list(range(len(df.sid.cat.categories)))
print "Index state:", df.index.is_monotonic
if not df.index.is_monotonic: df.sort_index(inplace=True)
df.head()

Index state: True


,sid,value
timestamp,,
2016-03-26 04:38:28+01:00,1,488.25
2016-03-26 04:38:38+01:00,1,490.00
2016-03-26 04:38:48+01:00,1,487.75
2016-03-26 04:38:58+01:00,1,489.25
2016-03-26 04:39:08+01:00,1,488.50


## Storing to disk
Note that for using select over rows later, the to-be-used column needs to be declared as a DataColumn

In [661]:
def store_df(df):
    with pd.HDFStore('../local/db/test_store.h5', complevel=9, complib='blosc') as store:
        store.put('data', df, format='table', data_columns=['sid'])
store_df(df)
print os.path.getsize('../local/db/test_store.h5')/2**10, 'KiB'

543 KiB


## Accessing from disk (simple)

In [662]:
store = pd.HDFStore('../local/db/test_store.h5')
print store
store.data.head()
store.data.value[store.data.sid==1].head()

<class 'pandas.io.pytables.HDFStore'>
File path: ../local/db/test_store.h5
/data                          frame_table  (typ->appendable,nrows->26336,ncols->2,indexers->[index],dc->[sid])
/data/meta/sid/meta            series_table (typ->appendable,nrows->3,ncols->1,indexers->[index],dc->[values]) 


timestamp
2016-03-26 04:38:28+01:00    488.25
2016-03-26 04:38:38+01:00    490.00
2016-03-26 04:38:48+01:00    487.75
2016-03-26 04:38:58+01:00    489.25
2016-03-26 04:39:08+01:00    488.50
Name: value, dtype: float16

## Accessing from disk (queries)

In [663]:
store = pd.HDFStore('../local/db/test_store.h5')
store.select('data', where="sid=1")
store.close


<bound method HDFStore.close of <class 'pandas.io.pytables.HDFStore'>
File path: ../local/db/test_store.h5
/data                          frame_table  (typ->appendable,nrows->26336,ncols->2,indexers->[index],dc->[sid])
/data/meta/sid/meta            series_table (typ->appendable,nrows->3,ncols->1,indexers->[index],dc->[values]) >

## Reading in as DataFrame

In [664]:
df = pd.read_hdf('../local/db/test_store.h5', 'data')
print df.dtypes
df.tail()


sid      category
value     float16
dtype: object


,sid,value
timestamp,,
2016-03-28 18:31:53+02:00,1,488.2500
2016-03-28 18:32:03+02:00,1,492.7500
2016-03-28 18:32:13+02:00,1,487.7500
2016-03-28 18:32:23+02:00,2,22.3125
2016-03-28 18:32:23+02:00,1,491.7500


## Appending rows to table (non-categorical sensor column!)

In [57]:
h5file = tables.open_file('../local/db/consolidation/test.h5', "a")
h5file.root.data.table.coldtypes


{'index': dtype('int64'),
 'nid': dtype('uint8'),
 'sid': dtype('uint8'),
 'values_block_0': dtype(('<f4', (1,)))}

In [60]:
row = h5file.root.data.table.row
for n in range(10000):
    row['index'] = pd.Timestamp.now().to_datetime64()
    row['values_block_0'] = n
    row['sid'] = 1
    row['nid'] = 0
    row.append()
h5file.root.data.table.flush()

In [29]:
with pd.HDFStore('../local/db/test_store.h5', 'r') as store:
    print store.get_storer('data').nrows
# doesn't work to get last row of node_id, methinks

26336


In [3]:
with pd.HDFStore('../local/db/telemetry.h5', 'r') as store:
    print store.select('data', where='nid==1').index[-1]
%timeit pd.read_hdf('../local/db/telemetry.h5', 'data')


2016-03-31 01:06:19.030053888
10 loops, best of 3: 21.7 ms per loop


In [12]:
df = pd.read_hdf('../local/db/telemetry.h5', 'data')
%timeit max(df.loc[df.sid==1].index)

10 loops, best of 3: 32.3 ms per loop


In [38]:
def t1(df):
    g = df.groupby('nid')
    return (max(g.get_group(0).index), max(g.get_group(1).index))
df = pd.read_hdf('../local/db/telemetry.h5', 'data')
%timeit t1(df)
print t1(df)

10 loops, best of 3: 110 ms per loop
(Timestamp('2016-03-31 01:06:09.240590080'), Timestamp('2016-03-31 01:06:19.030053888'))


In [39]:
def t2(df):
    df.sort_index(inplace=True)
    g = df.groupby('nid')
    return (g.get_group(0).index[-1], g.get_group(1).index[-1])
df = pd.read_hdf('../local/db/telemetry.h5', 'data')
%timeit t2(df)
print t2(df)

100 loops, best of 3: 8.82 ms per loop
(Timestamp('2016-03-31 01:06:09.240590080'), Timestamp('2016-03-31 01:06:19.030053888'))


In [40]:
def t3(df):
    return (max(df.loc[df.nid==0].index), max(df.loc[df.nid==1].index))
df = pd.read_hdf('../local/db/telemetry.h5', 'data')
%timeit t3(df)
print t3(df)

10 loops, best of 3: 105 ms per loop
(Timestamp('2016-03-31 01:06:09.240590080'), Timestamp('2016-03-31 01:06:19.030053888'))


In [50]:
def t4(df):
    df.sort_index(inplace=True)
    g = df.groupby('nid')
    return {k: g.get_group(k).index[-1] for k in [0, 1]}
df = pd.read_hdf('../local/db/telemetry.h5', 'data')
%timeit t4(df)
print t4(df)

100 loops, best of 3: 8.79 ms per loop
{0: Timestamp('2016-03-31 01:06:09.240590080'), 1: Timestamp('2016-03-31 01:06:19.030053888')}


In [51]:
### WTF?! 
def t5(df):
    df.sort_index(inplace=True)
    g = df.groupby('nid')
    return {k: g.get_group(k).index[-1] for k in g.groups}
df = pd.read_hdf('../local/db/telemetry.h5', 'data')
%timeit t5(df)
print t5(df)

1 loop, best of 3: 465 ms per loop
{0: Timestamp('2016-03-31 01:06:09.240590080'), 1: Timestamp('2016-03-31 01:06:19.030053888')}


In [55]:
### WTF?! 
def t6(df):
    df.sort_index(inplace=True)
    g = df.groupby('nid')
    return {k: g.get_group(k).index[-1] for k in set(df.nid)}
df = pd.read_hdf('../local/db/telemetry.h5', 'data')
%timeit t6(df)
print t6(df)
%timeit set(df.nid)

10 loops, best of 3: 20.3 ms per loop
{0: Timestamp('2016-03-31 01:06:09.240590080'), 1: Timestamp('2016-03-31 01:06:19.030053888')}
